# 第3章 データ構造

## 3.1 Pythonのネイティブなデータ構造

### 3.1.1 リスト

In [1]:
small_list = list(range(10))

In [2]:
%%timeit
last_element = small_list[-1]

8.83 ns ± 0.0273 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [3]:
large_list = list(range(10_000))

In [4]:
%%timeit
last_element = large_list[-1]

9.22 ns ± 0.0186 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [5]:
%%timeit
4200 in small_list  # 4200がsmall_listにあるか

43.4 ns ± 0.0261 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
%%timeit
4200 in large_list

16.5 μs ± 252 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### 3.1.2 タプル

### 3.1.3 辞書

In [7]:
from faker import Faker


fake = Faker()

In [8]:
small_dict = {}

for i in range(10):
    small_dict[fake.name()] = fake.address()
name = list(small_dict.keys())[0]  # ←先頭要素のキーを得る（他のキーでも可）
name

'Jason Ramos'

In [9]:
%%timeit
small_dict[name]  # ←実際にあるキーを入力する必要がある

8.28 ns ± 0.0637 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [10]:
large_dict = {}
for i in range(10_000):
    large_dict[fake.name()] = fake.address()
list(large_dict.keys())[0]
name = list(large_dict.keys())[0]
name

'Amanda Lane'

In [11]:
%%timeit
large_dict[name]

8.54 ns ± 0.0572 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


### 3.1.4 セット（集合）

In [12]:
%%timeit
4200 in large_dict

9.2 ns ± 0.127 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [13]:
%%timeit
large_set = set(large_list)
4200 in large_set

42.3 μs ± 390 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [14]:
%%timeit
large_set = set(large_list)
4592 in large_set

42.7 μs ± 672 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## 3.2 Numpy

### 3.2.1 NumPyの関数

従来の方法（リスト内包表記を使った場合）

```python
python_2d_list = [[1, 3, 5], [2, 4, 6], [7, 9, 11]]

first_column = [python_2d_list[i][0] for i in range(len(python_2d_list))]
```

In [15]:
# Numpyによる配列操作
import numpy as np


nd_2d_array = np.array([[1, 3, 5], [2, 4, 6], [7, 9, 11]])
first_column = nd_2d_array[:, 0]

### 3.2.2 NumPyのパフォーマンスに関する考察


```python
mixed_type_list = ["one", 2, 3.14]
```
Pythonのリストでは上のように文字列、整数、浮動小数点数を記憶できる。  
同じデータを記憶するためにNumPyを使って次のようなコードを書いてもエラーにはならない。

In [16]:
mixed_type_array = np.array(["one", 2, 3.14])

しかし、出力してみると、すべて文字列に変換されている。

In [17]:
print(mixed_type_array)

['one' '2' '3.14']


NumPyでは要素の型も一緒に保存される。

In [18]:
integer_array = np.array([1, 2, 3])
print(integer_array.dtype)

int64


この型は標準のPythonのintではなくint64になる。**Numpyでは標準的なPythonの型とは異なる型を用いており、これはパフォーマンスの向上にもつながる。**  
→ Numpyのすべての要素が同じ型であることから、特に**ベクトル化計算**（Vectorized Calculation）で、大きなパフォーマンス向上につながる。

通常のPythonのforループでは、Pythonインタプリタがどの関数を適用するべきかを知るために、すべての要素の型をチェックする必要がある。  

しかし、NumPyはこのステップをスキップして、多くの数値演算で最適化されたCコードを実行できる。**NumPyでは配列内のすべての要素を繰り返し演算するのではなく、一度にすべての要素を対象にして演算する**。  
→ これは**ベクトル化**（Vectorize）と呼ばれる。（ベクトル化による高速化の詳細は[Itamar Turner-Trauing](https://oreil.ly/DumcS)の記事に詳しい）  

In [19]:
random_int_array = np.random.randint(1, 100_000, 100_000)
random_int_list = list(random_int_array)

In [20]:
%%timeit -r 7 -n 100
sum(random_int_list)

1.46 ms ± 138 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
%%timeit -r 7 -n 100
np.sum(random_int_array)

12.5 μs ± 3.77 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
array_to_fill = np.zeros(1000)

In [23]:
random_int_array = np.random.randint(1, 100_000, 100_000)

In [24]:
random_int_array.nbytes

800000

In [25]:
random_int_array.dtype

dtype('int64')

In [26]:
random_int_array_32 = random_int_array.astype(np.int32)

In [27]:
random_int_array_32.nbytes

400000

In [28]:
small_array = np.array([1, 3, 5], dtype=np.int16)

### 3.2.3 Daskを使った配列演算

In [29]:
large_np_array = np.random.randint(1, 100000, 3_000_000_000)

In [30]:
%%timeit -r 1 -n 1
np.max(large_np_array)

51 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [31]:
import dask.array as da


large_dask_array = da.random.randint(1, 100000, 3_000_000_000)

In [32]:
large_dask_array = da.from_array(large_np_array)

In [33]:
%%timeit -r 1 -n 3
array_max = large_dask_array.max()
array_max.compute()

21.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)


In [34]:
from dask.distributed import Client


client = Client(n_workers=4)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 18.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51637,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:51650,Total threads: 3
Dashboard: http://127.0.0.1:51656/status,Memory: 4.50 GiB
Nanny: tcp://127.0.0.1:51640,


## 3.3 機械学習における配列

In [35]:
np_tensor = np.random.rand(4, 4)

In [36]:
import tensorflow as tf


tf_tensor = tf.convert_to_tensor(np_tensor)

In [37]:
import torch


torch_tensor = torch.from_numpy(np_tensor)

## 3.4 Pandas

# 3.4.1 DataFrameの機能

In [38]:
import pandas as pd


usa_data = pd.Series([13.33, 14.02, 14.02, 14.25], index=["2000", "2001", "2002", "2003"])

In [39]:
india_data = pd.Series([9.02, 9.01, 8.84, 8.84], index=["2000", "2001", "2002", "2003"])

df = pd.DataFrame({"USA": usa_data, "India": india_data})

### 3.4.2 DataFrameのパフォーマンス

In [40]:
%%timeit
df["India_fraction"] = df["India"] / 100

22.6 μs ± 628 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [41]:
%%timeit
df["India_franction"] = df["India"].apply(lambda x: x / 100)

22.8 μs ± 50.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [42]:
%%timeit
df["India_fraction"] = [row['India'] / 100 for index, row in df.iterrows()]

47.3 μs ± 3.79 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## 3.5 まとめ